In [1]:
from ccnet_spark.text_normalizer import normalize
from ccnet_spark.pipe_preprocess import load_segments
from ccnet_spark.pipe_hash import compute_hashes,split_doc2para
from ccnet_spark.pipe_lid import predictLang
from ccnet_spark.pipe_tokenized import doSentencePiece
from ccnet_spark.pipe_perplexity import doDocLM
from ccnet_spark.pipe_ppbucket import doPPBucket
from ccnet_spark.pipe_save import save_partation,load_partation
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.functions import sum as spark_sum

# 初始化 SparkSession
spark = SparkSession.builder.appName("CCNETSpark")  \
                    .config("spark.executor.memory", "64g") \
                    .config("spark.driver.memory", "32g") \
                    .config("spark.driver.maxResultSize", "32g") \
                    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
                    .getOrCreate()

24/04/02 15:43:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
def getModePara(mode):
    if(mode=="test"):
        n_segments=10
        cache_folder="/root/wxl_folder/cache_data/"
        date="2019-09" ## hardcode ,现在只能是这个
        segments=[i for i in range(n_segments)]
        min_len=300
        isSample=True
        sampleRate=0.01
        num_partitions=1
    else:
        n_segments=4
        cache_folder="/root/wxl_folder/cache_data/"
        date="2019-09" ## hardcode ,现在只能是这个
        segments=[i for i in range(n_segments)]
        min_len=300
        isSample=False
        sampleRate=1
        num_partitions=4
    return [cache_folder,date,segments,min_len,isSample,sampleRate,num_partitions]
mode="dev"
cache_folder,date,segments,min_len,isSample,sampleRate,num_partitions=getModePara(mode)

In [7]:
selected_bucket="tail"
selected_lang="en"
df_en_head=load_partation(spark,selected_lang,selected_bucket,cache_folder,date,isSample,sampleRate,min_len)
df_en_head.select("url","raw_content","perplexity","length","cc_segment").show()
print(df_en_head.count())

+--------------------+--------------------+----------+------+----------+
|                 url|         raw_content|perplexity|length|cc_segment|
+--------------------+--------------------+----------+------+----------+
|https://phetnakar...|mobile intermitte...|    3173.1| 10323|         0|
|http://www.graceu...|Telephone: 952-43...|     824.5|   740|         1|
|https://css.cdn.a...|Scotland, is it n...|     701.6| 10971|         1|
|https://www.defen...|Welcome to Defens...|    2133.2|  1129|         1|
|https://www.alber...|Port Alberni Blac...|     521.5|  4442|         2|
|http://paws2care....|Synergy 2\nZ Gall...|    8983.2|  1133|         2|
|https://phillysto...|$1,896.99 – $3,84...|     707.2|  3806|         0|
|http://nova-antiq...|German Beer Stein...|    1202.0|  2626|         2|
|https://www.anime...|No Bootleg Policy...|    1466.7|  1149|         0|
|http://roncarspea...|Other Guides\nNew...|     593.1|  2958|         0|
|https://tacanow.o...|Company name to b...|    1714